In [ ]:
!pip install pandas-plink limix-lmm

In [ ]:
import numpy as np
from pandas_plink import read_plink
from limix_lmm import LMM
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from statsmodels.stats.multitest import multipletests
import seaborn as sns

In [ ]:
import scipy
import scipy.stats as st

if not hasattr(scipy, 'dot'):
    scipy.dot = np.dot
if not hasattr(scipy, 'einsum'):
    scipy.einsum = np.einsum
if not hasattr(scipy, 'log'):
    scipy.log = np.log
if not hasattr(scipy, 'sign'):
    scipy.sign = np.sign
if not hasattr(scipy, 'sqrt'):
    scipy.sqrt = np.sqrt

### Preprocessing pipeline

BFILE=/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/ALL.chr22_GRCh38.genotypes.20170504

OUTDIR=/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/qc

mkdir -p "$OUTDIR"


plink --bfile "$BFILE" --freq --missing --out "$OUTDIR"/initial_qc


Preprocessing 

We only kept SNP like variants (single base substitutions), dropped variants with missing rate > 0.02 and minor allele frequency < 0.01 to keep only commons, and also variants that have extreme deviations that are caused by noise rather than signal

(filtered data set written as a new bed/bim/fam)


plink --bfile "$BFILE" \
  --snps-only \
  --geno 0.02 \
  --maf 0.01 \
  --hwe 1e-6 \
  --make-bed \
  --out "$OUTDIR"/chr22_step1_common


check if there are duplicate variant ids

plink --bfile "$OUTDIR"/chr22_step1_common \
  --list-duplicate-vars ids-only suppress-first \
  --out "$OUTDIR"/dupcheck

wc -l "$OUTDIR"/dupcheck.dupvar

there were no duplicates (0)

plink --bfile "$OUTDIR"/chr22_step1_common \
  --exclude "$OUTDIR"/dupcheck.dupvar \
  --make-bed \
  --out "$OUTDIR"/chr22_step2_nodup

sample level missingness, none was filtered (both chr22_step2_nodup.fam and chr22_step3_sampleqc.fam are same size 2504)

plink --bfile "$OUTDIR"/chr22_step2_nodup \
  --mind 0.02 \
  --make-bed \
  --out "$OUTDIR"/chr22_step3_sampleqc




INITIAL:

Number of variants: 109827 

Number of samples: 2504

AFTER QC:

Number of variants: 59743

Number of samples: 2504


In [ ]:
#bfile = '/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/qc/chr22_step3_sampleqc'
bfile = '/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/pca/before_qc/qc/chr22_step3_sampleqc'
bim, fam, G = read_plink(bfile)

In [ ]:
n_snps = bim.shape[0]
n_samples = fam.shape[0]
n_snps, n_samples

In [ ]:
X_real = G.compute().T

In [ ]:
X_real.shape

Standardize the genotype matrix, so that all SNPs are on the same scale, 3 variants were dropped, which had a standard deviation equals to 0 among all individuals. (bim annotation table must be also updated)

In [ ]:
#whole chromosome 22

mu_full = X_real.mean(axis=0)
sd_full = X_real.std(axis=0, ddof=0)
keep_full = sd_full > 1e-12
keep_idx  = np.where(keep_full)[0] 
standardized_X = (X_real[:, keep_full] - mu_full[keep_full]) / sd_full[keep_full]

In [ ]:
# bim data for variants that werent dropped after the standardization
bim_kept = bim.iloc[keep_idx].copy().reset_index(drop=True)

bim_kept["orig_bim_idx"] = keep_idx

In [ ]:
X_real.shape

In [ ]:
standardized_X[:, 4]

In [ ]:
standardized_X.shape

### Phase 1: Phenotype Simulation

First, we need to set the parameters (heritability and number of casual SNPs):

In [ ]:
BASE_SEED = 42

def seed_for(h2, m, base=BASE_SEED):
    h2_int = int(round(h2 * 1_000_000))  
    return base + 1_000_003 * h2_int + 10_007 * int(m)

In [ ]:

M = 10 
h2 = 0.6 

rng = np.random.default_rng(42)
idx_caus = rng.choice(standardized_X.shape[1], size=M, replace=False)
var_expl = np.repeat(h2/M, M)

Code from GWAS exercise to simulate phenotypes (maybe delete the standardization part from here since its been done beforehand):

In [ ]:
def simulate_pheno(X, idx_caus, var_expl, rng, direction=None):
    # Ensure that the number of causal variant indices matches the number of variances explained.
    assert len(idx_caus) == len(var_expl)

    # If no direction is provided, randomly assign a positive or negative direction for each causal variant.
    if direction is None:
        direction = 2. * (rng.random(len(idx_caus)) > 0.5) - 1.
    # Ensure that the number of directions matches the number of causal variant indices.
    assert len(idx_caus) == len(direction)

    # Compute the remaining variance after accounting for the variance explained by the causal variants.
    ve = 1 - var_expl.sum()
    # Ensure that the total variance explained by causal variants is less than 1.
    assert ve > 0, 'sum(var_expl) should be < 1'

    # Compute the effect sizes for the causal variants based on the variance they explain and their direction.
    beta = np.sqrt(var_expl) * direction

    # Extract the columns of X corresponding to the causal variants and standardize them.
    Xc = X[:, idx_caus]
    Xc = (Xc - Xc.mean(0)) / Xc.std(0)

    # Compute the genetic component of the phenotype.
    yg = Xc.dot(beta)[:, None]
    # Compute the noise component of the phenotype.
    yn = np.sqrt(ve) * rng.standard_normal((X.shape[0], 1))

    # Sum the genetic and noise components to get the simulated phenotype.
    y = yg + yn

    # Initialize the real effect sizes for all variants in X as zeros.
    beta_real = np.zeros(X.shape[1])
    # Update the real effect sizes for the causal variants.
    beta_real[idx_caus] = beta

    # Standardize the phenotypic values to have mean 0 and standard deviation 1.
    ystd = y.std()
    y = (y - y.mean()) / ystd
    # Adjust the real effect sizes accordingly after standardizing y.
    beta_real = beta_real / ystd

    return y, beta_real

In [ ]:
def qq_plot(p_values, title):
    """
    Create a QQ plot given a list of p-values.

    Parameters:
    - p_values: list of p-values
    - title: title for the plot
    """

    # Sort p-values
    observed = -np.log10(np.sort(p_values))
    expected = -np.log10(np.arange(1, len(p_values) + 1) / (len(p_values) + 2))

    # Create the QQ plot
    plt.scatter(expected, observed, marker='.')
    plt.plot([0, max(expected)], [0, max(expected)], color='red', linestyle='--')
    plt.xlabel('Expected -log10(P-value)')
    plt.ylabel('Observed -log10(P-value)')
    plt.title(title)

Apply the code:

In [ ]:
y, beta_real = simulate_pheno(standardized_X, idx_caus, var_expl, rng)

In [ ]:
y.shape

##### LD Pruning for PCA & PCA

plink --bfile data/final/chr22_step3_sampleqc \
  --indep-pairwise 200 50 0.2 \
  --out chr22_prune

plink --bfile data/final/chr22_step3_sampleqc \
  --extract chr22_prune.prune.in \
  --pca 10 \
  --out chr22_pca

##### 1) build phenotype table with IDs

In [ ]:
pheno = fam[["fid","iid"]].copy()
pheno.columns = ["FID","IID"]
pheno["y"] = y.reshape(-1)

##### 2) load PCs and merge

In [ ]:
pcs = pd.read_csv("/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/pca/chr22_pca.eigenvec", sep=r"\s+", header=None, engine="python")
pcs.columns = ["FID","IID"] + [f"PC{i}" for i in range(1, pcs.shape[1]-1)]

df = pheno.merge(pcs, on=["FID","IID"], how="inner", validate="one_to_one")

##### 3) build F (intercept + PCs) for LMM

In [ ]:
k = 10
F = np.column_stack([np.ones((df.shape[0], 1)),
                     df[[f"PC{i}" for i in range(1, k+1)]].to_numpy()])

### Phase 2: GWAS / Feature Selection

Now we have:

- our genotype matrix **`standardized_X`** which was first standardized (before that it was **`X_real`**) and then used to simulate the phenotype matrix
- our simulated phenotype matrix **`y`** 
- our covariates (PCs) **`F`**

#### 1. Training/Validation Set Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, F_train, F_test = train_test_split(
    standardized_X, y, F,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

In [ ]:
X_train.shape, y_train.shape, F_train.shape

#### 2. GWAS

GWAS is only peformed on the training set (training genotype matrix **`X_train`** and phenotype matrix **`y_train`**)

In [ ]:
lmm = LMM(y_train, F_train)
lmm.process(X_train)
pv = lmm.getPv()
beta = lmm.getBetaSNP()
beta_ste = lmm.getBetaSNPste()

In [ ]:
pv.shape

In [ ]:
np.isnan(pv).any(), np.isnan(pv).sum(), np.where(np.isnan(pv)) 

In [ ]:
qq_plot(pv, 'QQ training')
plt.show()

In [ ]:
x = bim_kept['pos'].values
plt.subplot(211)
plt.title('Real effect size')
plt.plot(x, beta_real, '.k')
plt.ylabel('eff size')
plt.subplot(212)
plt.title('GWAS results')
plt.plot(x, -np.log10(pv), '.k')
plt.ylabel('-log$_{10}$ P')
plt.tight_layout()
plt.show()

#### 3. Clumping using PLINK 

First create a file with SNP ids and their corresponding p-values

In [ ]:
gwas_for_clump = pd.DataFrame({
    "SNP": bim_kept["snp"].astype(str),   
    "P": np.asarray(pv, dtype=float)
})
gwas_for_clump.to_csv("gwas_for_clump.txt", sep="\t", index=False)

clump-p1: p-val threshold for index SNPs
clump-p2: secondary p val threshold for SNPs
clump-r2: LD threshold for joining a SNP to an index SNPs clump 
clump-kb: window radius around the index SNP 

can try playing around with the parameter

plink \
  --bfile data/final/chr22_step3_sampleqc \
  --clump gwas_for_clump.txt \
  --clump-snp-field SNP \
  --clump-field P \
  --clump-p1 1e-4 \
  --clump-p2 1e-2 \
  --clump-r2 0.1 \
  --clump-kb 250 \
  --out data/chr22_clumped


plink \
  --bfile data/final/chr22_step3_sampleqc \
  --clump gwas_for_clump.txt \
  --clump-snp-field SNP \
  --clump-field P \
  --clump-p1 0.1 \
  --clump-r2 0.1 \
  --clump-kb 500 \
  --out chr22_clumped


try these: --clump-p1 1 --clump-r2 0.1 --clump-kb 500 

In [ ]:
# snp IDs used in GWAS (aligned with pv)
snps_gwas = set(bim["snp"].astype(str))   

# SNP IDs in PLINK reference
bim_ref = pd.read_csv("data/qc/chr22_step3_sampleqc.bim", sep=r"\s+", header=None)
bim_ref.columns = ["chrom","snp","cm","pos","a1","a2"]
snps_ref = set(bim_ref["snp"].astype(str))

missing = snps_gwas - snps_ref
print("missing in plink ref:", len(missing))

In [ ]:
clumped = pd.read_csv("/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/chr22_clumped.clumped", sep=r"\s+", engine="python")
lead_snps = clumped["SNP"].astype(str).tolist()

In [ ]:
len(lead_snps), lead_snps[:5]

In [ ]:
snp_to_col = pd.Series(index=bim_kept["snp"].astype(str), data=range(bim_kept.shape[0]))
cols = snp_to_col.reindex(lead_snps).dropna().astype(int).to_numpy()
cols


In [ ]:
# Which causal variants made it into your lead SNPs?
causal_snp_names = set(bim_kept.iloc[idx_caus]["snp"].astype(str))
recovered = causal_snp_names.intersection(set(lead_snps))
print(f"Directly recovered: {len(recovered)}/10 causal variants")

In [ ]:
# For each causal variant NOT in lead_snps, find its LD with lead SNPs
causal_not_recovered = causal_snp_names - set(lead_snps)

# Check if any lead SNP is within the clumping window (250kb) of these causal variants
for snp in causal_not_recovered:
    causal_pos = bim_kept[bim_kept["snp"] == snp]["pos"].values[0]
    nearby_leads = [ls for ls in lead_snps 
                    if abs(bim_kept[bim_kept["snp"] == ls]["pos"].values[0] - causal_pos) < 250000]
    print(f"{snp}: {len(nearby_leads)} lead SNPs within 250kb")

In [ ]:
X_clump_train = X_train[:, cols] 
X_clump_test = X_test[:, cols]   

### Phase 3: Model Development

Train a regression model to predict the phenotype based on the selected SNPs and test it on the
validation set

In [ ]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error


**`X_clump_train`**: training genotype matrix only including clumped SNPs

**`X_clump_test`**: test genotype matrix only including clumped SNPs

**`y_train`**: training phenotypes

**`y_test`**: test phenotypes


In [ ]:
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_clump_train, y_train)

y_pred = regr.predict(X_clump_test)

R2: close to 1 ~ good performance

MSE: close to 0 ~ good performance 

spearman r: 
- close to -1 ~ monotonic negative correlation
- close to 1 ~ monotonic positive correlation


In [ ]:
from scipy import stats
spearman = stats.spearmanr(y_test, y_pred)
print("R2", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("spearman r: ", spearman.statistic)

In [ ]:
def plot_prediction(y_pred, input, model):
    plt.scatter(y_test, y_pred)

    xx = np.linspace(y_pred.min(), y_pred.max(), 200)

    plt.plot(xx, xx, linewidth=2, linestyle="--", label="ideal: y=x", color="orange")
    plt.title(f"{model} with {input}")
    plt.xlabel("Phenotype true")
    plt.ylabel("Phenotype predicted")
    plt.show()

In [ ]:
plot_prediction(y_pred, input="Causal Variants: 10 heritability: 0.6", model="Linear Regression")

In [ ]:
X_train_w_pcs = np.hstack([X_clump_train, F_train[:, 1:]])  # F[:, 1:] excludes intercept
X_test_w_pcs = np.hstack([X_clump_test, F_test[:, 1:]])

regr.fit(X_train_w_pcs, y_train)
y_pred_w_pcs = regr.predict(X_test_w_pcs)

In [ ]:
spearman_w_pcs = stats.spearmanr(y_test, y_pred_w_pcs)
print("R2", r2_score(y_test, y_pred_w_pcs))
print("MSE:", mean_squared_error(y_test, y_pred_w_pcs))
print("spearman r: ", spearman_w_pcs.statistic)

In [ ]:
plot_prediction(y_pred_w_pcs, input="Causal Variants: 10 heritability: 0.6 including PCs as Covariates", model="Linear Regression")

#### Predict with Multi Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(
    hidden_layer_sizes=(8, 16),
    activation="relu",
    max_iter=1000,
    alpha=0.001,
    beta_1=0.5,
    random_state=42
    )

In [ ]:
mlp.fit(X_train_w_pcs, y_train)
y_pred_w_mlp = mlp.predict(X_test_w_pcs)

In [ ]:
spearman_w_mlp = stats.spearmanr(y_test, y_pred_w_mlp)
print("R2", r2_score(y_test, y_pred_w_mlp))
print("MSE:", mean_squared_error(y_test, y_pred_w_mlp))
print("spearman r: ", spearman_w_mlp.statistic)

In [ ]:
plot_prediction(y_pred_w_mlp, input="Causal Variants: 10 heritability: 0.6 including PCs as Covariates", model="MLP")